In [1]:
# 1. 데이터 로드
import pandas as pd
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
           'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
           'hours-per-week', 'native-country', 'income']
data = pd.read_csv(url, header=None, names=columns, na_values='?', skipinitialspace=True)

In [2]:
data = data.dropna()

QR 방법을 사용한 이상치 제거 가이드

IQR 계산
- IQR(Interquartile Range)은 데이터의 중앙 50%를 나타내며, 데이터의 변동성을 측정하는 데 사용됩니다. IQR은 3사분위수(Q3)와 1사분위수(Q1)의 차이로 계산됩니다.
  - Q1: 데이터의 25번째 백분위수(1사분위수)
  - Q3: 데이터의 75번째 백분위수(3사분위수)
  - IQR: Q3 - Q1

이상치 경계 계산
- 일반적으로 IQR의 1.5배를 사용하여 이상치 경계를 설정합니다.
  - Lower Bound: Q1 - 1.5 * IQR
  - Upper Bound: Q3 + 1.5 * IQR
- 이 경계를 벗어나는 데이터 포인트는 이상치로 간주됩니다.

이상치를 제거는 데이터의 분포를 왜곡하는 극단적인 값을 제거하여 데이터 분석 및 모델 성능을 향상시키는 데 도움이 됩니다. 도메인 지식과 데이터의 특성을 고려해서 이상치 제거를 진행합니다.

In [3]:
#이상치 제거

Q1 = data['capital-gain'].quantile(0.25)
Q3 = data['capital-gain'].quantile(0.75)
IQR = Q3 - Q1
capital_gain_outlier = data[(data['capital-gain'] < Q1 - 1.5 * IQR) | (data['capital-gain'] > Q3 + 1.5 * IQR)]

Q1 = data['capital-loss'].quantile(0.25)
Q3 = data['capital-loss'].quantile(0.75)
IQR = Q3 - Q1
capital_loss_outlier = data[(data['capital-loss'] < Q1 - 1.5 * IQR) | (data['capital-loss'] > Q3 + 1.5 * IQR)]

data = data.drop(capital_gain_outlier.index)
data = data.drop(capital_loss_outlier.index)

In [5]:
# 파생 변수 작성
data['capital_diff'] = data['capital-gain'] - data['capital-loss']

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26197 entries, 1 to 32559
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             26197 non-null  int64 
 1   workclass       26197 non-null  object
 2   fnlwgt          26197 non-null  int64 
 3   education       26197 non-null  object
 4   education-num   26197 non-null  int64 
 5   marital-status  26197 non-null  object
 6   occupation      26197 non-null  object
 7   relationship    26197 non-null  object
 8   race            26197 non-null  object
 9   sex             26197 non-null  object
 10  capital-gain    26197 non-null  int64 
 11  capital-loss    26197 non-null  int64 
 12  hours-per-week  26197 non-null  int64 
 13  native-country  26197 non-null  object
 14  income          26197 non-null  object
 15  capital_diff    26197 non-null  int64 
dtypes: int64(7), object(9)
memory usage: 3.4+ MB


One-Hot Encoding
- 비순서형 범주형 변수: 대부분의 범주형 변수는 순서가 없습니다. 예를 들어, workclass, occupation, race, sex 등의 변수는 순서가 없습니다. 이 경우, 각 범주를 고유한 이진 벡터로 변환하는 것이 적합합니다.
- 모델의 가정: Logistic Regression과 같은 선형 모델은 입력 변수들이 서로 독립적이라고 가정합니다. One-hot encoding은 이러한 가정을 유지하는 데 도움이 됩니다.
- 다중공선성 방지: One-hot encoding은 다중공선성 문제를 피하기 위해 첫 번째 범주를 제거할 수 있습니다. 이 방법은 drop_first=True 옵션을 사용하여 구현됩니다.

Label Encoding
- 순서형 범주형 변수: Label encoding은 범주에 순서가 있는 경우에 적합합니다. 예를 들어, 교육 수준(예: 초등학교 < 중학교 < 고등학교 < 대학교)은 순서가 있는 범주형 변수입니다.
- 트리 기반 모델: 결정 트리, 랜덤 포레스트, XGBoost와 같은 트리 기반 모델은 label encoding된 변수를 더 잘 처리할 수 있습니다. 이는 트리 기반 모델이 변수 간의 순서를 처리할 수 있기 때문입니다.

Adult Income 데이터셋의 경우, 대부분의 범주형 변수는 순서가 없으므로 one-hot encoding을 사용하는 것이 적절

In [7]:
# 범주형 변수 인코딩
categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country','income']
data = pd.get_dummies(data, columns=categorical_features, drop_first=True)

In [8]:
X = data.drop('income_>50K',axis=1)
y=data['income_>50K']

In [10]:

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# 데이터 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 학습용과 테스트용 데이터셋으로 나누기
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Logistic Rogression 모델 생성 및 학습
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [11]:
# 예측 및 평갸
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)


print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

Accuracy: 0.851526717557252
Confusion Matrix:
 [[3943  264]
 [ 514  519]]
Classification Report:
               precision    recall  f1-score   support

       False       0.88      0.94      0.91      4207
        True       0.66      0.50      0.57      1033

    accuracy                           0.85      5240
   macro avg       0.77      0.72      0.74      5240
weighted avg       0.84      0.85      0.84      5240

